In [3]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix
from imblearn.over_sampling import SMOTE
import warnings
warnings.filterwarnings('ignore')

# Load the CICIDS2017 dataset (Tuesday file with FTP/SSH brute force)
data = pd.read_csv('data/Tuesday-WorkingHours.pcap_ISCX.csv')
data.columns

Index([' Destination Port', ' Flow Duration', ' Total Fwd Packets',
       ' Total Backward Packets', 'Total Length of Fwd Packets',
       ' Total Length of Bwd Packets', ' Fwd Packet Length Max',
       ' Fwd Packet Length Min', ' Fwd Packet Length Mean',
       ' Fwd Packet Length Std', 'Bwd Packet Length Max',
       ' Bwd Packet Length Min', ' Bwd Packet Length Mean',
       ' Bwd Packet Length Std', 'Flow Bytes/s', ' Flow Packets/s',
       ' Flow IAT Mean', ' Flow IAT Std', ' Flow IAT Max', ' Flow IAT Min',
       'Fwd IAT Total', ' Fwd IAT Mean', ' Fwd IAT Std', ' Fwd IAT Max',
       ' Fwd IAT Min', 'Bwd IAT Total', ' Bwd IAT Mean', ' Bwd IAT Std',
       ' Bwd IAT Max', ' Bwd IAT Min', 'Fwd PSH Flags', ' Bwd PSH Flags',
       ' Fwd URG Flags', ' Bwd URG Flags', ' Fwd Header Length',
       ' Bwd Header Length', 'Fwd Packets/s', ' Bwd Packets/s',
       ' Min Packet Length', ' Max Packet Length', ' Packet Length Mean',
       ' Packet Length Std', ' Packet Length Variance', '

In [6]:

# Strip whitespace from column names (common issue in CICIDS2017 CSV)
data.columns = data.columns.str.strip()

# Select relevant features for brute force detection
features = [
    'Destination Port', 'Flow Duration', 'Total Fwd Packets',
    'Total Backward Packets', 'Fwd Packet Length Mean', 'Bwd Packet Length Mean',
    'Flow Packets/s', 'Flow IAT Mean', 'Fwd IAT Total', 'Bwd IAT Total',
    'Fwd PSH Flags', 'SYN Flag Count', 'ACK Flag Count'
]

# Target column
target = 'Label'

# Clean the dataset
# Replace infinite values with NaN
data.replace([np.inf, -np.inf], np.nan, inplace=True)
# Drop rows with NaN values
data.dropna(subset=features + [target], inplace=True)

# Encode categorical labels (Benign, FTP-Patator, SSH-Patator)
le = LabelEncoder()
data['Label'] = le.fit_transform(data[target])

# Split features and target
X = data[features]
y = data['Label']

# Split into training and testing sets (80% train, 20% test)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

# Handle class imbalance using SMOTE
smote = SMOTE(random_state=42)
X_train_res, y_train_res = smote.fit_resample(X_train, y_train)

# Scale numerical features
scaler = StandardScaler()
X_train_res = scaler.fit_transform(X_train_res)
X_test = scaler.transform(X_test)

# Initialize and train Random Forest Classifier
rf_model = RandomForestClassifier(n_estimators=100, random_state=42, n_jobs=-1)
rf_model.fit(X_train_res, y_train_res)

# Predict on test set
y_pred = rf_model.predict(X_test)

# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred, average='weighted')
recall = recall_score(y_test, y_pred, average='weighted')
f1 = f1_score(y_test, y_pred, average='weighted')
conf_matrix = confusion_matrix(y_test, y_pred)

# Print evaluation metrics
print("Model Performance Metrics:")
print(f"Accuracy: {accuracy:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1-Score: {f1:.4f}")
print("\nConfusion Matrix:")
print(conf_matrix)
print("\nClass Labels:", le.classes_)

# Feature importance
feature_importance = pd.DataFrame({
    'Feature': features,
    'Importance': rf_model.feature_importances_
}).sort_values(by='Importance', ascending=False)
print("\nFeature Importance:")
print(feature_importance)

Model Performance Metrics:
Accuracy: 0.9998
Precision: 0.9998
Recall: 0.9998
F1-Score: 0.9998

Confusion Matrix:
[[86346     0    17]
 [    0  1587     0]
 [    5     0  1174]]

Class Labels: ['BENIGN' 'FTP-Patator' 'SSH-Patator']

Feature Importance:
                   Feature  Importance
0         Destination Port    0.424095
4   Fwd Packet Length Mean    0.132552
2        Total Fwd Packets    0.085526
5   Bwd Packet Length Mean    0.067279
3   Total Backward Packets    0.061237
8            Fwd IAT Total    0.053035
1            Flow Duration    0.051523
6           Flow Packets/s    0.033826
9            Bwd IAT Total    0.026725
7            Flow IAT Mean    0.025243
11          SYN Flag Count    0.021499
10           Fwd PSH Flags    0.012622
12          ACK Flag Count    0.004840


In [9]:
from sklearn.metrics import classification_report
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00     86363
           1       1.00      1.00      1.00      1587
           2       0.99      1.00      0.99      1179

    accuracy                           1.00     89129
   macro avg       1.00      1.00      1.00     89129
weighted avg       1.00      1.00      1.00     89129



In [10]:
from sklearn.metrics import confusion_matrix
print(confusion_matrix(y_test, y_pred))

[[86346     0    17]
 [    0  1587     0]
 [    5     0  1174]]
